**CAGR Annual_rate of return** :- average annual rate over a period of time 

**Cumulative return** :- total return percentage of gain or loss on an investment 

**Sharpe ratio** :- its a financial metric used to measure the investment risk adjusted return by assessing how much excess return an investment generates per unit of risk 

**Track Drawdown** :- Its a systematically monitor and measure the decline in the value of investment from its highest point to a subsequent lowest point before a new peak is reached 



In [1]:
from IPython.display import display
import pandas as pd 
import yfinance as yf 
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import minimize

portfolio = ["CANBK.NS", "BEL.NS", "ADANIPOWER.NS", "ITC.NS", "ASHOKLEY.NS", "LTF.NS", "BHEL.NS", "BIOCON.NS", "SAIL.NS", "BANKINDIA.NS", "IREDA.NS","INDUSTOWER.NS", "SWIGGY.NS", "NYKAA.NS","FEDERALBNK.NS", "AARTIIND.NS"]
df_port = yf.download(portfolio, period="1y", auto_adjust=False)["Adj Close"]
df_port.head(5)
px.line(df_port).show()


#Analysis risk and return 

def portfolio_analysis(df_port, risk_free_rate):
    # Returns
    log_returns = np.log(df_port / df_port.shift(1)).dropna()

    # Daily volatility (decimal)
    daily_volatility = log_returns.std()
    port_daily_volatility = daily_volatility.mean()
    percentage_port_daily_volatility = port_daily_volatility * 100

    # Annual volatility (decimal)
    annual_volatility = daily_volatility * np.sqrt(df_port.shape[0])
    port_annual_volatility = annual_volatility.mean()
    percentage_port_annual_volatility = port_annual_volatility * 100

    # CAGR (percent)
    cagr_annual_return = ((df_port.iloc[-1] / df_port.iloc[0])**(1/len(df_port)) - 1) * 100
   

    # Cumulative return (decimal)
    cumulative_return = (df_port.iloc[-1] - df_port.iloc[0]) / df_port.iloc[0]
    port_cumulative_return = cumulative_return.mean()
    percentage_cumulative_return = cumulative_return * 100
    percentage_port_cumulative_return = port_cumulative_return * 100


    # Sharpe Ratio
    risk_free_rate = risk_free_rate
    portfolio_return_mean_ = (log_returns.mean() * df_port.shape[0])- risk_free_rate
    sharpe_ratio = portfolio_return_mean_ / annual_volatility
    show_graph_df = sharpe_ratio.reset_index()
    show_graph_df.columns = ['Fund', 'Sharpe_Ratio']

    # Correlation matrix
    corr = df_port.corr()

    # Drawdown
    rolling_max = df_port.cummax()
    drawdown = (df_port - rolling_max) / rolling_max
    max_drawdown = drawdown.min() * 100  

    show_graph=px.line(df_port).show()
    corr_graph = px.imshow(corr, text_auto=True, aspect="auto", color_continuous_scale='RdBu', title=' Post Optimization Correlation Matrix').show()
    pie_chart = px.pie(show_graph_df,     values='Sharpe_Ratio',  # Column for the size of each slice
    names='Fund',           # Column for the labels of each slice
    title='Portfolio Sharpe Ratios') # Optional: Add a title)
    return print(f"daily_volatility of the portfolio:\n{percentage_port_daily_volatility}%\n"), print(f"Annual volatility of the portfolio:\n{percentage_port_annual_volatility}%\n"), print(f"Average cumlative_returnin percentage:\n{percentage_port_cumulative_return}%\n"), print(f"Average CAGR in percentage:\n{cagr_annual_return.mean()}%\n"), print(f"Average Sharpe Ratio:\n{sharpe_ratio.mean()}\n"),  print(f"Average Max Drawdown:\n{max_drawdown.mean()}%\n"), print(f"sharpe ratio for each stock {sharpe_ratio}"), print(f"Max Draw Down for eaxh stock{max_drawdown}"), corr_graph, print(f"Cumulative return for each stock in the portfolio in percentage:{display(percentage_cumulative_return)}%\n"),show_graph, pie_chart.show()


[*********************100%***********************]  16 of 16 completed


In [2]:
portfolio_analysis(df_port, risk_free_rate=0.0653) # as per 16 November 2025 the risk free rate in india is 6.53%

daily_volatility of the portfolio:
2.062201112619698%

Annual volatility of the portfolio:
32.540984677688996%

Average cumlative_returnin percentage:
31.84205245596622%

Average CAGR in percentage:
0.09860636923939153%

Average Sharpe Ratio:
0.5733755453684708

Average Max Drawdown:
-24.015500280127178%

sharpe ratio for each stock Ticker
AARTIIND.NS     -0.473116
ADANIPOWER.NS    0.629532
ASHOKLEY.NS      0.875515
BANKINDIA.NS     1.033566
BEL.NS           1.257619
BHEL.NS          0.533192
BIOCON.NS        0.594081
CANBK.NS         1.331204
FEDERALBNK.NS    0.507809
INDUSTOWER.NS    0.577022
IREDA.NS        -0.723397
ITC.NS          -0.960534
LTF.NS           2.338166
NYKAA.NS         1.406467
SAIL.NS          0.511962
SWIGGY.NS       -0.265081
dtype: float64
Max Draw Down for eaxh stockTicker
AARTIIND.NS     -24.438772
ADANIPOWER.NS   -19.803916
ASHOKLEY.NS     -16.930704
BANKINDIA.NS    -23.639285
BEL.NS          -22.643899
BHEL.NS         -29.377217
BIOCON.NS       -24.679361
CAN

Ticker
AARTIIND.NS       -8.091863
ADANIPOWER.NS     45.468345
ASHOKLEY.NS       38.442028
BANKINDIA.NS      48.147813
BEL.NS            53.890216
BHEL.NS           28.486068
BIOCON.NS         29.556730
CANBK.NS          57.326220
FEDERALBNK.NS     20.103457
INDUSTOWER.NS     26.922482
IREDA.NS         -19.629904
ITC.NS            -9.732054
LTF.NS           119.337909
NYKAA.NS          58.371139
SAIL.NS           26.896633
SWIGGY.NS         -6.022380
dtype: float64

Cumulative return for each stock in the portfolio in percentage:None%



(None, None, None, None, None, None, None, None, None, None, None, None)

In [3]:
#portfolio optimization 

more_stocks = ["IDFCFIRSTB.NS", "PNB.NS", "AUBANK.NS", "HDFCBANK.NS", "ICICIBANK.NS", "BAJAJFINSV.NS", "CIPLA.NS", "DRREDDY.NS" ,"IPCALAB.NS", "LUPIN.NS", "ZYDUSLIFE.NS", "SBICARD.NS", "ASIANPAINT.NS", "HINDUNILVR.NS", "TCS.NS", "INFY.NS", "LT.NS", "MARUTI.NS", "SUNPHARMA.NS", "WIPRO.NS", "ADANIGREEN.NS", "ADANIPOWER.NS",     "DLF.NS", "DMART.NS", "DRREDDY.NS", "EIDPARRY.NS",
    "EICHERMOT.NS", "EIHOTEL.NS", "ELGIEQUIP.NS", "EMAMILTD.NS", "FEDERALBNK.NS",
    "GAIL.NS", "GLAND.NS", "GLAXO.NS", "GLENMARK.NS", "GNFC.NS",
    "GODREJAGRO.NS", "GODREJCP.NS", "GODREJPROP.NS", "GRANULES.NS", "GRASIM.NS",
    "HAL.NS", "HAVELLS.NS", "HCLTECH.NS", "HDFCAMC.NS", "HDFCBANK.NS",
    "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ICICIGI.NS", "ICICIBANK.NS",
    "IEX.NS", "INDHOTEL.NS", "INDIGO.NS", "INDOWIND.NS", "INDUSTOWER.NS",
    "INDUSINDBK.NS", "INFY.NS", "INOXGREEN.NS", "INOXWIND.NS",
    "IOC.NS", "IPCALAB.NS", "IRCTC.NS", "IRFC.NS", "ITC.NS",
    "JINDALSTEL.NS", "JSWENERGY.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "KPIGREEN.NS",
    "LAURUSLABS.NS", "LTIM.NS", "LT.NS", "LUPIN.NS", "M&M.NS",
    "MANKIND.NS", "MARICO.NS", "MARUTI.NS", "MAXHEALTH.NS",
    "MOREPENLAB.NS", "MOTHERSON.NS", "MPHASIS.NS", "NATCOPHARM.NS", "NAUKRI.NS",
    "NESTLEIND.NS", "NHPC.NS", "NMDC.NS", "NTPC.NS",
    "OBEROIRLTY.NS", "OLECTRA.NS", "ONGC.NS", "ORCHPHARMA.NS",
    "PAGEIND.NS", "PAYTM.NS", "PFC.NS", "PFIZER.NS", "PNB.NS",
    "PNBHOUSING.NS", "POLICYBZR.NS", "POWERGRID.NS", "RECLTD.NS", "RELAXO.NS",
    "RELIANCE.NS", "SANOFI.NS", "SBIN.NS", "SBILIFE.NS",
    "SHREECEM.NS", "SIEMENS.NS", "SUNPHARMA.NS",
    "SUZLON.NS", "SWSOLAR.NS", "TATACONSUM.NS", "TATAPOWER.NS", "TATASTEEL.NS",
    "TCS.NS", "TECHM.NS", "TITAN.NS", "TORNTPHARM.NS", "TORNTPOWER.NS",
    "TRENT.NS",  "UPL.NS", "VBL.NS", "VEDL.NS",
    "VGUARD.NS", "WESTLIFE.NS",
    "WHIRLPOOL.NS", "WIPRO.NS", "YESBANK.NS", "ZYDUSLIFE.NS"]
portfolio.extend(more_stocks)

df_port = yf.download(portfolio, period="1y", auto_adjust=False)["Adj Close"]


[*********************100%***********************]  127 of 127 completed


In [4]:
# stock selection and optimization

benchmark_sharpe =  0.999
optimized_stocks = []

# Volatility for each stock 
log_returns = np.log(df_port / df_port.shift(1)).dropna()
daily_volatility = log_returns.std()
annual_volatility_per = (daily_volatility * np.sqrt(df_port.shape[0])) * 100 
annual_volatility= (daily_volatility * np.sqrt(df_port.shape[0]))
# Cumulative return (decimal per stock)
cumulative_return = (df_port.iloc[-1] - df_port.iloc[0]) / df_port.iloc[0]
percentage_cumulative_return = cumulative_return * 100

#sharpe ratio
risk_free_rate = 0.0653
portfolio_return_mean_ = (log_returns.mean() * df_port.shape[0]) - risk_free_rate
sharpe_ratio = portfolio_return_mean_ / annual_volatility


# Combined filter using & operator
filtered = sharpe_ratio < benchmark_sharpe

# Extract ticker names
optimized_stocks = list(filtered[filtered].index)

# Remove these from portfolio
portfolio = [stock for stock in portfolio if stock not in optimized_stocks]

# Download updated portfolio data
df_port = yf.download(portfolio, period="1y", auto_adjust=False)["Adj Close"]

print("Removed:", optimized_stocks)
print("Updated & Downloaded portfolio:", portfolio)


[*********************100%***********************]  13 of 13 completed


Removed: ['AARTIIND.NS', 'ADANIGREEN.NS', 'ADANIPOWER.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'BAJAJFINSV.NS', 'BHEL.NS', 'BIOCON.NS', 'CIPLA.NS', 'DLF.NS', 'DMART.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'FEDERALBNK.NS', 'GAIL.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GNFC.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRASIM.NS', 'HAL.NS', 'HAVELLS.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'INDHOTEL.NS', 'INDOWIND.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXGREEN.NS', 'INOXWIND.NS', 'IOC.NS', 'IPCALAB.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'JINDALSTEL.NS', 'JSWENERGY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'KPIGREEN.NS', 'LT.NS', 'LTIM.NS', 'LUPIN.NS', 'M&M.NS', 'MANKIND.NS', 'MAXHEALTH.NS', 'MOREPENLAB.NS', 'MOTHERSON.NS', 'MPHASIS.NS', 'NATCOPHARM.NS', 'NAUKRI.NS', 'NESTLEIND.NS', 'NHPC.NS', 'N

In [5]:
portfolio_analysis(df_port, risk_free_rate=0.0653) # as per 16 November 2025 the risk free rate in india is 6.53%

daily_volatility of the portfolio:
1.8196224324109846%

Annual volatility of the portfolio:
28.713157669207757%

Average cumlative_returnin percentage:
60.91052074554823%

Average CAGR in percentage:
0.18685566831613606%

Average Sharpe Ratio:
1.3826837660966758

Average Max Drawdown:
-18.49345670447065%

sharpe ratio for each stock Ticker
AUBANK.NS        1.252374
BANKINDIA.NS     1.033566
BEL.NS           1.257619
CANBK.NS         1.331204
EICHERMOT.NS     1.231217
INDIGO.NS        1.265764
LAURUSLABS.NS    1.913234
LTF.NS           2.338166
MARICO.NS        0.964584
MARUTI.NS        1.499420
NYKAA.NS         1.406467
PAYTM.NS         1.158676
UPL.NS           1.322598
dtype: float64
Max Draw Down for eaxh stockTicker
AUBANK.NS       -19.231408
BANKINDIA.NS    -23.639285
BEL.NS          -22.643895
CANBK.NS        -26.239968
EICHERMOT.NS    -14.254590
INDIGO.NS       -15.449737
LAURUSLABS.NS   -18.694942
LTF.NS          -13.463307
MARICO.NS       -14.883882
MARUTI.NS       -13.776309


Ticker
AUBANK.NS         57.828262
BANKINDIA.NS      48.147813
BEL.NS            53.890216
CANBK.NS          57.326220
EICHERMOT.NS      41.256238
INDIGO.NS         47.949721
LAURUSLABS.NS    111.741853
LTF.NS           119.337909
MARICO.NS         29.776185
MARUTI.NS         44.672123
NYKAA.NS          58.371139
PAYTM.NS          70.034441
UPL.NS            51.504650
dtype: float64

Cumulative return for each stock in the portfolio in percentage:None%



(None, None, None, None, None, None, None, None, None, None, None, None)

In [6]:
# Markowite portfolio model

tickers = df_port.columns.tolist()
num_assets = len(tickers)

returns = df_port.pct_change().dropna()
mean_returns = returns.mean() * len(df_port)        # Annualized expected return
std_returns = returns.std() * np.sqrt(len(df_port)) # Annualized risk (std dev)
cov_matrix = returns.cov() * len(df_port)           # Annualized covariance matrix

# -----------------------------
#  Portfolio functions
# -----------------------------
def portfolio_return(weights):
    return np.sum(weights * mean_returns)

def portfolio_variance(weights):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

bounds = tuple((0, 1) for _ in range(num_assets))

target_returns = np.linspace(mean_returns.min(), mean_returns.max(), 100)
risks = []
weights_record = []

for r in target_returns:
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: portfolio_return(x) - r}
    )
    result = minimize(portfolio_variance, num_assets * [1. / num_assets],
                      method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success:
        risks.append(np.sqrt(result.fun))
        weights_record.append(result.x)

# -----------------------------
#  Convert to percentage form
# -----------------------------
risks_percent = np.array(risks) * 100
returns_percent = np.array(target_returns) * 100

# -----------------------------
#  DataFrame for individual stocks
# -----------------------------
stock_df = pd.DataFrame({
    'Ticker': [ticker.replace('.NS', '') for ticker in tickers],
    'Expected Return (%)': (mean_returns * 100).round(2),
    'Risk (Volatility %)': (std_returns * 100).round(2)
})

# -----------------------------
# 5️⃣ Plotly Visualization
# -----------------------------
fig = go.Figure()

# Efficient Frontier
fig.add_trace(go.Scatter(
    x=risks_percent, y=returns_percent,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='crimson', width=4, dash='solid'),
    hovertemplate='<b>Risk:</b> %{x:.2f}%<br><b>Return:</b> %{y:.2f}%<extra></extra>'
))

# Individual Stocks
fig.add_trace(go.Scatter(
    x=std_returns * 100, y=mean_returns * 100,
    mode='markers+text',
    textposition='top center',
    marker=dict(size=11, color='royalblue', line=dict(width=1, color='white')),
    name='Individual Stocks',
    hovertemplate='<b>%{text}</b><br>Risk: %{x:.2f}%<br>Return: %{y:.2f}%<extra></extra>'
))

# -----------------------------
#  Layout Aesthetic Improvements
# -----------------------------
fig.update_layout(
    title=dict(text='<b> Pre optimization Efficient Frontier with Individual Stocks</b>', x=0.5, font=dict(size=22)),
    xaxis=dict(title='Risk (Standard Deviation %)', tickformat='.1f'),
    yaxis=dict(title='Expected Return (%)', tickformat='.1f'),
    template='plotly_white',
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(family='Verdana', size=13),
    hoverlabel=dict(bgcolor='white', font_size=13)
)

fig.show()

# -----------------------------
#  Styled DataFrame Display
# -----------------------------
stock_df.style.background_gradient(cmap='RdYlGn', subset=['Expected Return (%)']) \
         .background_gradient(cmap='Blues', subset=['Risk (Volatility %)'])


,Ticker,Expected Return (%),Risk (Volatility %)
Ticker,,,
AUBANK.NS,AUBANK,50.790000,31.670000
BANKINDIA.NS,BANKINDIA,44.570000,32.020000
BEL.NS,BEL,47.590000,29.350000
CANBK.NS,CANBK,49.810000,29.300000
EICHERMOT.NS,EICHERMOT,37.320000,22.960000
INDIGO.NS,INDIGO,42.710000,25.950000
LAURUSLABS.NS,LAURUSLABS,81.850000,35.710000
LTF.NS,LTF,83.780000,31.170000
MARICO.NS,MARICO,28.250000,20.420000
